# Data Mining Project - MS data

Sean Deloddere - 914691 <br>
Giulio Marcon - 

## Load in data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

In [2]:
ms_df = pd.read_csv('msdata.csv', index_col = 0)
ms_df.head()

,class,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f4991,f4992,f4993,f4994,f4995,f4996,f4997,f4998,f4999,f5000
id,,,,,,,,,,,,,,,,,,,,,
1,1,15.9163,14.9321,15.0390,17.8641,13.9613,13.7340,16.0623,18.8601,13.2050,...,14.6288,15.3938,16.5050,15.5140,15.3419,16.1521,13.5364,17.3797,11.0531,17.6632
2,1,16.7356,15.4755,16.3641,17.1888,15.6816,13.8570,15.3676,18.7065,13.1914,...,14.3400,16.9335,13.9464,15.4398,15.3828,14.6178,14.3196,19.1234,11.6904,18.9757
3,1,16.0818,15.5128,16.0463,17.7098,16.3100,14.0004,15.2811,18.2759,14.3015,...,14.4362,17.3463,13.8546,15.2012,15.1175,15.4171,14.4186,18.3941,11.5052,18.2264
4,1,16.5298,14.4695,15.0169,18.9366,17.2052,14.0647,16.7984,16.8349,13.2690,...,14.2350,15.6478,12.4729,16.1781,14.4310,15.7018,13.4911,13.3618,9.5018,17.4410
5,1,16.6449,15.2317,16.1317,16.6674,14.0423,13.5085,15.2629,18.2195,12.3015,...,14.0427,17.0514,14.9944,15.1685,15.0767,15.1256,14.0295,19.6029,9.1163,18.4286


In [3]:
ms_cols_with_missing = [col for col in ms_df.columns if ms_df[col].isnull().any()]

print("ms dataframe has "+str(len(ms_cols_with_missing))+" columns with missing values.")

ms dataframe has 0 columns with missing values.


In [4]:
if len(ms_df.columns) == 5001:
    true_labels = ms_df['class']
    ms_df = ms_df.drop('class',axis=1)

In [5]:
ms = ms_df.to_numpy()
ms = ms.astype(np.float)
print(ms)

[[15.9163 14.9321 15.039  ... 17.3797 11.0531 17.6632]
 [16.7356 15.4755 16.3641 ... 19.1234 11.6904 18.9757]
 [16.0818 15.5128 16.0463 ... 18.3941 11.5052 18.2264]
 ...
 [16.8768 14.7009 16.0331 ... 18.0713  7.4234 17.4883]
 [16.6324  8.714  15.4801 ... 19.1017  5.4156 10.7977]
 [16.5564  9.7216 15.4208 ... 18.1179 10.9165 17.3568]]


## Pre-processing

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import DistanceMetric

In [7]:
ss = StandardScaler()
ms_ss = ss.fit_transform(ms)

In [8]:
mas = MaxAbsScaler()
ms_mas = mas.fit_transform(ms)

In [9]:
ms_cs = cosine_similarity(ms)
ms_cs_ss = cosine_similarity(ms_ss)
ms_cs_mas = cosine_similarity(ms_mas)

In [10]:
def euclidean_similarity(data):
    return 1/(1+euclidean_distances(data, data))

In [11]:
ms_ed = euclidean_similarity(ms)
ms_ed_ss = euclidean_similarity(ms_ss)
ms_ed_mas = euclidean_similarity(ms_mas)

## Clustering

In [12]:
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering

In [13]:
def get_nmi(labels1,labels2):
    return normalized_mutual_info_score(labels1,labels2,average_method='geometric')

### K-Means

In [14]:
def perform_KMeans(data):
    kmeans_list = []
    for i in range(1,9):
        kmeans = KMeans(n_clusters=i).fit(data)
        labels = kmeans.labels_
        kmeans_list.append((i,get_nmi(true_labels,labels),labels))
    sorted_kmeans_list = sorted(kmeans_list, key=lambda x:x[1], reverse=True)
    return sorted_kmeans_list

In [15]:
kmeans_0 = perform_KMeans(ms)
kmeans_ss = perform_KMeans(ms_ss)
kmeans_mas = perform_KMeans(ms_mas)

In [34]:
for i in range(3):
    print(i)
    print(kmeans_0[i][0:2])
    print(kmeans_ss[i][0:2])
    print(kmeans_mas[i][0:2])

0
(2, 0.4990403989991282)
(1, 0.25)
(1, 0.25)
1
(4, 0.35245772002943493)
(8, 0.150240566482656)
(4, 0.23039634362671738)
2
(3, 0.2724439235397712)
(3, 0.11296394108074036)
(6, 0.16927990512548)


### Hierarchical

In [17]:
def perform_hierarchical(data,linkage):
    hier_list = []
    for i in range(1,9):
        hier = AgglomerativeClustering(n_clusters=i,linkage=linkage).fit(data)
        labels = hier.labels_
        hier_list.append((i,get_nmi(true_labels,labels),labels))
    sorted_hier_list = sorted(hier_list, key=lambda x:x[1], reverse=True)
    return sorted_hier_list        

In [18]:
linkages = ['ward','complete','average','single']
hier_0 = []
hier_ss = []
hier_mas = []
for linkage in linkages:
    print(linkage)
    hier_0.append((linkage,perform_hierarchical(ms,linkage)))
    hier_ss.append((linkage,perform_hierarchical(ms_ss,linkage)))
    hier_mas.append((linkage,perform_hierarchical(ms_mas,linkage)))

ward
complete
average
single


In [37]:
for i in range(4):
    print("============================================")
    print(linkages[i])
    print("0")
    for j in range(3):
        print(hier_0[i][1][j][0:2])
    print("ss")
    for j in range(3):
        print(hier_ss[i][1][j][0:2])
    print("mas")
    for j in range(3):
        print(hier_mas[i][1][j][0:2])

ward
0
(5, 0.4739938347242577)
(6, 0.44632441019560576)
(7, 0.4277016120569969)
ss
(6, 0.3840490366198332)
(7, 0.3695913426247361)
(8, 0.360991476291801)
mas
(4, 0.5826743412077233)
(5, 0.5444413723904372)
(6, 0.5052375371794963)
complete
0
(8, 0.2683424053873574)
(1, 0.25)
(5, 0.24801554690137462)
ss
(7, 0.255719325551191)
(8, 0.2507309209089276)
(1, 0.25)
mas
(1, 0.25)
(7, 0.24785083275381253)
(6, 0.24457616340611107)
average
0
(1, 0.25)
(8, 0.03572245741022434)
(7, 0.03302978442800275)
ss
(1, 0.25)
(8, 0.03572245741022434)
(7, 0.03302978442800275)
mas
(1, 0.25)
(8, 0.03572245741022434)
(7, 0.03302978442800275)
single
0
(1, 0.25)
(8, 0.03572245741022434)
(7, 0.03302978442800275)
ss
(1, 0.25)
(8, 0.03467767612162472)
(7, 0.0319232585145711)
mas
(1, 0.25)
(8, 0.03572245741022434)
(7, 0.03302978442800275)


### Spectral 

#### Nearest Neighbor

In [38]:
def perform_spectral_nn(data):
    spectral_list = []
    for nc in range(2,8):
        for nn in range(5,9,2):
            spectral = SpectralClustering(n_clusters=nc, affinity='nearest_neighbors',n_neighbors=nn,n_jobs=3).fit(data)
            labels = spectral.labels_
            spectral_list.append((nc,nn,get_nmi(true_labels,labels),labels))
    sorted_spectral_list = sorted(spectral_list, key=lambda x:x[2], reverse=True)
    return sorted_spectral_list 

In [39]:
spectral_0 = perform_spectral_nn(ms)
spectral_ss = perform_spectral_nn(ms_ss)
spectral_mas = perform_spectral_nn(ms_mas)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [48]:
print("0")
for i in range(3):
    print(spectral_0[i][0:3])

print("ss")
for i in range(3):
    print(spectral_ss[i][0:3])

print("mas")
for i in range(3):
    print(spectral_mas[i][0:3])

0
(4, 5, 0.042587356028537034)
(4, 7, 0.042587356028537034)
(5, 5, 0.022330756636793427)
ss
(4, 5, 0.042587356028537034)
(4, 7, 0.028135934702172953)
(5, 5, 0.020422713584982714)
mas
(4, 5, 0.042587356028537034)
(4, 7, 0.03238275974538477)
(5, 5, 0.020517634177940405)


#### RBF

In [41]:
def perform_spectral_rbf(data):
    spectral_list = []
    for nc in range(2,8):
        spectral = SpectralClustering(n_clusters=nc, affinity='rbf',n_jobs=3).fit(data)
        labels = spectral.labels_
        spectral_list.append((nc,get_nmi(true_labels,labels),labels))
    sorted_spectral_list = sorted(spectral_list, key=lambda x:x[1], reverse=True)
    return sorted_spectral_list 

In [42]:
spectral_rbf_0 = perform_spectral_rbf(ms)
spectral_rbf_ss = perform_spectral_rbf(ms_ss)
spectral_rbf_mas = perform_spectral_rbf(ms_mas)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:310: RuntimeWarning: invalid value encountered in add
  distances += XX
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:310: RuntimeWarning: invalid value encountered in add
  distances += XX
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\u

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:199: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:310: RuntimeWarning: invalid value encountered in add
  distances += XX
C:\ProgramData\Anaconda3\lib\s

In [49]:
print("0")
for i in range(3):
    print(spectral_rbf_0[i][0:2])

print("ss")
for i in range(3):
    print(spectral_rbf_ss[i][0:2])

print("mas") 
for i in range(3):
    print(spectral_rbf_mas[i][0:2])

0
(4, 0.004900356098585696)
(5, 0.004332404501692476)
(7, 0.004117085444675135)
ss
(3, 0.024012905986220582)
(2, 0.013397951755189421)
(4, 0.013397951755189421)
mas
(7, 0.0889278247033067)
(6, 0.07320650890868385)
(5, 0.06366889915118204)


#### Precomputed

In [44]:
def perform_spectral_pc(data):
    spectral_list = []
    for nc in range(2,8):
        spectral = SpectralClustering(n_clusters=nc, affinity='precomputed',n_jobs=3).fit(data)
        labels = spectral.labels_
        spectral_list.append((nc,get_nmi(true_labels,labels),labels))
    sorted_spectral_list = sorted(spectral_list, key=lambda x:x[1], reverse=True)
    return sorted_spectral_list 

In [45]:
spectral_pc_cs = perform_spectral_pc(ms_cs)
#spectral_pc_cs_ss = perform_spectral_pc(ms_cs_ss)
spectral_pc_cs_mas = perform_spectral_pc(ms_cs_mas)
spectral_pc_ed = perform_spectral_pc(ms_ed)
spectral_pc_ed_ss = perform_spectral_pc(ms_ed_ss)
spectral_pc_ed_mas = perform_spectral_pc(ms_ed_mas)

In [51]:
print("cs 0")
for i in range(3):
    print(spectral_pc_cs[i][0:2])
#print(spectral_pc_cs_ss[0][0:2])
print("cs mas")
for i in range(3):
    print(spectral_pc_cs_mas[i][0:2])
print("ed 0")
for i in range(3):
    print(spectral_pc_ed[i][0:2])
print("ed ss")
for i in range(3):
    print(spectral_pc_ed_ss[i][0:2])
print("ed mas")
for i in range(3):
    print(spectral_pc_ed_mas[i][0:2])

cs 0
(3, 0.8477183741661061)
(2, 0.7156116773234342)
(5, 0.671250298706406)
cs mas
(3, 0.8748134474580916)
(2, 0.7435389119552804)
(4, 0.6831262295287711)
ed 0
(6, 0.8802085025252647)
(7, 0.8578779551735857)
(3, 0.5823011762355861)
ed ss
(6, 0.9085154581352097)
(7, 0.7172188470269978)
(5, 0.5782035367443347)
ed mas
(5, 0.9150261357661318)
(6, 0.8871018272034555)
(7, 0.7293143430854032)


# BEST METHOD

MaxAbsScaler + Euclidean Similarity + Spectral Clustering (5 clusters)

In [29]:
Best_labels = spectral_pc_ed_mas[0][2]
print(Best_labels)

[2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 2 4 2 4 3 4 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 2 4 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [30]:
with open('ms_solution.txt','w') as file:
    for label in Best_labels:
        file.write(str(label)+'\n')